# Problemas de clasificación
## Introducción a las Ciencias Sociales Computacionales
## Facultad de Ciencias Sociales, Universidad de Buenos Aires, 2023
### Juan Manuel Pérez y Rodolfo Elbert

En esta notebook vamos a realizar análisis muy simples sobre noticias 

In [1]:
import spacy

nlp = spacy.load('es_core_news_sm')

In [3]:
ds = load_dataset("finiteautomata/prueba-arg", split="train")

ds

Found cached dataset parquet (/users/jmperez/.cache/huggingface/datasets/finiteautomata___parquet/finiteautomata--prueba-arg-1b304d042a671975/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


Dataset({
    features: ['tweet_id', 'text', 'title', 'url', 'user', 'body', 'created_at', 'comments'],
    num_rows: 73423
})

In [4]:
# Me quedo con notas con > 20 comentarios

ds = ds.filter(lambda x: len(x['comments']) > 20)

ds

Filter:   0%|          | 0/73423 [00:00<?, ? examples/s]

In [5]:
ds

Dataset({
    features: ['tweet_id', 'text', 'title', 'url', 'user', 'body', 'created_at', 'comments'],
    num_rows: 38464
})

In [6]:
title = "La Corte Suprema de Justicia de la Nación declaró inconstitucional el decreto de necesidad y urgencia (DNU) que estableció la suspensión de las clases presenciales en el Área Metropolitana de Buenos Aires (AMBA)"
print(title)
entities = nlp(title).ents

entities

La Corte Suprema de Justicia de la Nación declaró inconstitucional el decreto de necesidad y urgencia (DNU) que estableció la suspensión de las clases presenciales en el Área Metropolitana de Buenos Aires (AMBA)


(Corte Suprema de Justicia de la Nación,
 DNU,
 Área Metropolitana de Buenos Aires,
 AMBA)

In [7]:
ds = ds.shuffle(seed=42).select(range(5_000))

In [8]:
titles = ds["title"]

In [9]:
from tqdm.auto import tqdm

# SpaCy pipe

entities = []

for title in tqdm(titles):
    if title is not None:
        entities.append(nlp(title).ents)
    else:
        entities.append([])

  0%|          | 0/5000 [00:00<?, ?it/s]

In [10]:

for title, entity_list in zip(titles, entities):
    if title:

        print(title)
        print(entity_list)

El kirchnerismo cierra filas en apoyo de Amado Boudou y refuerza su ataque a la Corte Suprema
(Amado Boudou, Corte Suprema)
El dardo del Beto Alonso a Mauricio Macri: “Debe haber ido a Paraguay a pedir los puntos de la final de Madrid”
(Beto Alonso, Mauricio Macri, “Debe, Paraguay, Madrid)
Eduardo Duhalde repitió su célebre frase: “La Argentina está condenada al éxito”
(Eduardo Duhalde, La Argentina)
Donald Trump fue nominado al premio Nobel de la Paz por impulsar el acuerdo entre Israel y Emiratos Árabes Unidos
(Donald Trump, Nobel de la Paz, Israel, Emiratos Árabes Unidos)
Por el proyecto del aborto, los obispos no saludarán al Presidente por la Navidad
(Presidente por la Navidad,)
Sergio Massa: “Hay que llevar al Parlamento y a la democracia argentina a la era digital del siglo XXI”
(Sergio Massa: “Hay, Parlamento)
Una encuesta explica el “desacople”: ¿por qué si la economía se derrumba la gente lo ve tan bien a Alberto Fernández?
(“, Alberto Fernández)
Ángel de Brito dio positivo d

In [11]:
# Contemos las entidades
from collections import Counter

text_entities = [entity.text for entity_list in entities for entity in entity_list]

Counter(text_entities).most_common(50)

[('Alberto Fernández', 429),
 ('“', 423),
 ('Coronavirus', 278),
 ('Gobierno', 201),
 ('Argentina', 194),
 ('Cristina Kirchner', 136),
 ('la Argentina', 108),
 ('Mauricio Macri', 99),
 ('coronavirus', 84),
 ('Axel Kicillof', 80),
 ('Hacemos', 78),
 ('Presidente', 61),
 ('Patricia Bullrich', 50),
 ('Horacio Rodríguez Larreta', 49),
 ('qué', 49),
 ('China', 43),
 ('Ciudad', 43),
 ('Vicentin', 42),
 ('Venezuela', 42),
 ('Cristina', 40),
 ('Sputnik V', 39),
 ('Macri', 37),
 ('Sergio Berni', 34),
 ('Uruguay', 33),
 ('Justicia', 33),
 ('Brasil', 33),
 ('Diego Maradona', 32),
 ('Pfizer', 32),
 ('Ginés González García', 31),
 ('Santiago Cafiero', 30),
 ('Formosa', 29),
 ('Juan Grabois', 29),
 ('Congreso', 29),
 ('Senado', 28),
 ('Carla Vizzotti', 26),
 ('Máximo Kirchner', 26),
 ('Marcelo Tinelli', 26),
 ('Donald Trump', 25),
 ('Cambio', 24),
 ('Amado Boudou', 23),
 ('La Argentina', 23),
 ('Jair Bolsonaro', 23),
 ('Estados Unidos', 23),
 ('Maradona', 23),
 ('año', 23),
 ('Rodríguez Larreta', 22

## Clasificación de comentarios

Vamos a intentar medir la polaridad de los comentarios de un diario. Para eso, vamos a usar la librería `transformers` de huggingface. Esta librería nos permite usar modelos de lenguaje pre-entrenados para hacer tareas de NLP. 

En este caso, vamos a medir la polaridad de los comentarios usando el modelo [`pysentimiento/robertuito-sentiment-analysis](https://huggingface.co/pysentimiento/robertuito-sentiment-analysis)

**NOTA**: Pueden usar directamente el paquete `pysentimiento` para hacer este análisis. Pero vamos a hacerlo "bajo nivel" para que vean cómo funciona.

In [13]:
df = ds.to_pandas()

df.head(2)

,tweet_id,text,title,url,user,body,created_at,comments
0,1336143617761681413,El kirchnerismo cierra filas en apoyo de Boudo...,El kirchnerismo cierra filas en apoyo de Amado...,https://bit.ly/2InSdw6,LANACION,"Ante el silencio de Alberto Fernández, los sec...",2020-12-08T03:00:37Z,"[{'created_at': '2020-12-08T03:02:43Z', 'text'..."
1,1282865815264600064,El dardo del Beto Alonso a Mauricio Macri: “De...,El dardo del Beto Alonso a Mauricio Macri: “De...,https://www.infobae.com/deportes-2/2020/07/14/...,infobae,“Macri debe haber ido a Paraguay a pedir los p...,2020-07-14T02:33:39Z,"[{'created_at': '2020-07-14T02:34:28Z', 'text'..."


In [15]:
df_comentarios = df.explode("comments")

df_comentarios["comment"] = df_comentarios["comments"].apply(lambda x: x["text"])

del df_comentarios["comments"]

df_comentarios.shape

(503472, 9)

In [16]:
pd.options.display.max_colwidth = 1000

df_comentarios.sample(10)[["title", "comment"]]

,title,comment
3153,"Coronavirus en Estados Unidos: al agravarse la pandemia, Donald Trump busca a quien culpar",@clarincom A los chinos
2087,"Desde este miércoles, los centros de diálisis no admitirán nuevos pacientes de PAMI",@clarincom Hdrp
4830,Le otorgaron la libertad al jubilado que mató a un ladrón en Quilmes: tuvieron que internarlo por una descompensación,@infobae Un asesino en libertad. Así estamos.
4781,La homofobia de Ofelia Fernández,"@laderechadiario Todos: fué hace años, la mina tenia 12 años era inconciente... Ñañaña\nMacho, ahora no tiene ni 20 años y puede proponer y votar por leyes que nos afectan a todos, alguien que no esta minimamente calificada para proponer algo por sobre los ciudadanos."
2835,Escándalo. Una diputada nacional debió pedir licencia tras difundirse que se quedaba con parte del sueldo de sus empleados,@LANACION Qlp!
3731,Apertura de sesiones 2021: el increíble despacho kirchnerista de Mayra Mendoza,"@LANACION ""Increíble""."
3771,"Mi historia con Néstor Kirchner, el mejor presidente que tuvo la democracia",@infobae https://t.co/cHTp3vMTLg
4583,Patricia Bullrich realizó una autocrítica sobre su militancia en la JP y aseguró: “Al Padre Mugica lo mataron los Montoneros”,"@clarincom Aaahh mirá che, la agrupación que ella integraba es la responsable de asesinar a Mugica. Y parece que la afirmación es para exonerar a los paramilitares de la triple A? Son de manual con la info, se les nota mucho para dónde van!!"
4466,Hebe de Bonafini contra Sergio Berni: “No puede pensar igual que Patricia Bullrich y justificar que te maten en el piso”,"@clarincom Perez Esquivel y esta señora son exponentes de los derechos humanos, está todo dicho, todo es una porquería."
3674,"“Ya pusimos el cuerpo, ahora que los diputados se pongan las pilas”: en el Congreso, el lado verde se prepara para una fiesta",@infobae Esas minas son de la coperativa le pagan 2mil x campaña para ir ahi y a donde sea que los llamen


## Preprocesamiento

Vamos a preprocessar los comentarios para que sean más fáciles de analizar. Para eso, vamos la función `preprocess_tweet` de `pysentimiento` que hace lo siguiente:

- Reemplaza los links por la palabra `url`
- Reemplaza los usuarios por la palabra `@usuario`
- Reemplaza los hashtags por la palabra `hashtag` e intenta partirlos en lo posible

In [17]:
from pysentimiento.preprocessing import preprocess_tweet

preprocess_tweet("RT @clarincom: Hola, esto es un tweet de prueba #EstoEsUnHashtag 😎 https://t.co/prueba")

'RT @usuario: Hola, esto es un tweet de prueba hashtag esto es un hashtag  emoji cara sonriendo con gafas de sol emoji  url'

In [18]:
from tqdm.auto import tqdm
# TQDM para ver el progreso
tqdm.pandas()
df_comentarios["comment"] = df_comentarios["comment"].progress_apply(preprocess_tweet)

  0%|          | 0/503472 [00:00<?, ?it/s]

In [19]:
df_comentarios.sample(10)[["title", "comment"]]

,title,comment
206,None,@usuario Por que no viene a c5n así le preguntan lo que nosotros queremos saber
3628,None,@usuario Love.
3920,Alberto Fernández volvió a insistir con que el mérito no es el camino para el crecimiento,"@usuario @usuario y para que tomas exámenes en la facultad, si el merito no es el camino???. TÍTERE, MULO!"
3754,Axel Kicillof: “Desde que Mauricio Macri volvió con su librito ha venido a hacer una oposición salvaje”,@usuario Jjaja jaja
4735,SOLICITADA. El periodista no es la fuente,@usuario url
2300,Érica Rivas hizo una grave denuncia contra Guillermo Francella: “Me dijo feminazi”,@usuario Otra loca del orto.
4877,"Juana Viale: ""Alberto Fernández parece más un vocero que la persona que toma decisiones""",@usuario Y vos??? Q pareces?...no tenés ni ahí la presencia para estar sentada ahí
1849,"Coparticipación. Sorprendido y enojado, Horacio Rodríguez Larreta evalúa llevar el tema a la Corte",@usuario @usuario A que no lo lleva
2700,Alberto Fernández: “Muchas veces el que delinque lo hace porque no encuentra una alternativa mejor”,@usuario Cínico Farsante.\nhashtag el peor presidente de la historia
2649,Debate: ¿Es cuestionable ir a vacunarse contra el coronavirus en el exterior?,"@usuario Ningún debate posible. En un país donde se afanan vacunas para dárselas a gente afín al gobierno de turno, todo aquel que quiere y puede, que lo haga. Dejen de alimentar el resentimiento."


In [42]:
model_name = "pysentimiento/robertuito-sentiment-analysis"

# Cargo pipeline de HuggingFace

from transformers import pipeline

pipe = pipeline("sentiment-analysis", model=model_name, tokenizer=model_name, device=0, batch_size=256, max_length=64)

pipe("SON UNOS HDP!!!")

[{'label': 'NEG', 'score': 0.8912988305091858}]

In [47]:
df_subset = df_comentarios.sample(30_000, random_state=42).copy()

del df_subset["body"]

preds = pipe(df_subset["comment"].values.tolist())

In [48]:
# Agrego columnas al dataframe

df_subset["preds"] = [pred["label"] for pred in preds]
df_subset["score"] = [pred["score"] for pred in preds]

df_subset["preds"].value_counts()

NEG    19315
NEU     8819
POS     1866
Name: preds, dtype: int64

In [53]:
df_subset[df_subset["preds"] == "NEU"].sample(10)[["text", "comment", "preds", "score"]]

,text,comment,preds,score
4582,Comenzó la infusión de Emmita 💪🏻♥️ https://t.co/ev41km8grH https://t.co/IDlEAp3CYS,@usuario,NEU,0.491545
3796,"Soledad Acuña: “Uno de los grandes problemas de la educación argentina es la formación docente, eligen militar en vez de enseñar” https://t.co/DtUVxsQw35 https://t.co/7WAJ3XMCE6","@usuario Alguien prestó atención a la cantidad de libros que tiene detrás suyo? Bueno, eso explica todo",NEU,0.705282
3,Donald Trump fue nominado al premio Nobel de la Paz por impulsar el acuerdo entre Israel y Emiratos Árabes Unidos https://t.co/ANXMPxUdOQ,@usuario emoji bíceps flexionado emoji emoji manos aplaudiendo emoji emoji manos aplaudiendo emoji,NEU,0.613570
1391,"La oposición salió a criticar la intervención de Vicentin: ""Es una señal trágica"" https://t.co/1S39nofQX1",@usuario Y si hubiera devuelto la guita y @usuario no lo tapara por ahí quien te dice estaría \nlibre de culpa y cargo,NEU,0.655994
3763,"En Córdoba, el banderazo contra del Gobierno nacional se hizo sentir con fuerza https://t.co/KLDR0qFdM2",@usuario Libertad de prensa ? O libertad de extorcion url,NEU,0.651880
2685,🔵ANTICIPO | La AFA decidió que el fútbol vuelva a Fase 1 por el aumento de los contagios https://t.co/NORG5b3DwP,@usuario Y... si 🤷‍,NEU,0.602525
2801,"Hebe de Bonafini sobre Ginés González García: ""No es buena persona "" https://t.co/g44rErWm6x https://t.co/SlhwvVyA60",@usuario Y Borges???,NEU,0.729223
3211,"Santiagueño falleció después de comer locro, empanadas, tomar vino y hacer el amor desenfrenadamente #UyTremendo #PasóMilVecesEsto #Peeero #NuncaConLocroYVino #DetallesYMás https://t.co/9b4LCbn7wV",@usuario url,NEU,0.482296
1039,"Mauricio Macri: “Cuando vino la crisis, a las 7 u 8 de la noche llegaba a Olivos, cerraba todo y ponía Netflix” https://t.co/DdkQf2oIPE","@usuario Si Mauri, nos dimos cuenta de eso. Es más, pensamos que ponías Netflix mucho más temprano.",NEU,0.664770
1385,Susana Giménez tiene coronavirus https://t.co/4LEY8qR1l3 https://t.co/eCqnxrxA6l,@usuario url,NEU,0.482296


In [56]:
df_subset[df_subset["preds"] == "POS"].sample(10)[["text", "comment", "preds", "score"]]

,text,comment,preds,score
3618,"Hugo Moyano: ""Estoy vacunado porque soy el presidente de dos obras sociales "" https://t.co/VGg8fk3GA1 https://t.co/2oXl0FxbAw",@usuario Sos indispensable de nada idiota,POS,0.536452
3112,Tensión en el Senado: Cristina Kirchner le cortó el micrófono a Esteban Bullrich luego de un duro cruce con José Mayans https://t.co/uPPC1xYRhZ,@usuario Apoyo total a Bullrich !!!,POS,0.591970
3059,"Insólita defensa de Oscar Parrilli a Lázaro Báez: “Lo condenaron por ser morocho, es un fallo racista” https://t.co/FkmTXF0w4I",@usuario Una respuesta a la altura de su electorado incondicional,POS,0.616354
676,Por la disparada del dólar libre se necesitan sólo USD 5.600 para ser millonario en pesos https://t.co/1I6QKSx093,@usuario Uuuhhhj soy millonario en pesos! Puedo comprar la provincia que quiera url,POS,0.691747
1576,Alberto Fernández entregó premios a Madres y Abuelas de Plaza de Mayo con un guiño a Cristina Kirchner https://t.co/F77UzSLVkf,@usuario Sigue la kermesse !!! Iujuuu,POS,0.807019
584,"Elecciones en EE.UU. El llamado urgente de Greta: ""Que todos voten por Biden "" https://t.co/Kz9Aw0KVXv https://t.co/fqfFBOTuyD",@usuario Jaja ya esta sabiendo que Trump ya tiene la Victoria emoji cara sonriendo con ojos sonrientes emoji,POS,0.733369
1088,El análisis político de Julián Serrano que revolucionó las redes: “La idea es hacerte bien pobre para que después le mendigues pan y arroz al Gobierno” https://t.co/OjU9hYR8jN,@usuario Guau tremendo análisis político,POS,0.872234
2780,Las vacunas no llegan y los votos se van https://t.co/IidtPZ5qYh,@usuario La nazzion siempre junto al pueblo. Jjaja,POS,0.831003
4692,Evo Morales volverá a la Argentina en la madrugada de este domingo después de un misterioso viaje relámpago a Venezuela https://t.co/3YUikLej6j,"@usuario Vamos Evo, le ganaste la elección a la Embajada. emoji cara sonriendo con ojos sonrientes emoji emoji manos aplaudiendo emoji emoji manos aplaudiendo emoji emoji manos aplaudiendo emoji",POS,0.594353
4205,Internaron a Eduardo Feinmann por coronavirus https://t.co/AM7CwDNACn,"@usuario FUERZA FEINMANN AVANTI ,QUE SE MEJORE PRONTO",POS,0.712629


## Ejercicios:


1. Intentar hacer un gráfico de comentarios positivos y negativos según fecha.
2. Intentar hacer un gráfico de comentarios positivos y negativos según entidad mencionada.
3. Trabajar con emociones en vez de polaridad. ¿Qué emociones predominan en los comentarios?